In [2]:
!pip install git+https://github.com/facebookresearch/ImageBind.git
!pip install torch torchvision torchaudio scikit-learn


  Cloning https://github.com/facebookresearch/ImageBind.git to /tmp/pip-req-build-oy2q5y2w
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/ImageBind.git /tmp/pip-req-build-oy2q5y2w
  Resolved https://github.com/facebookresearch/ImageBind.git to commit 53680b02d7e37b19b124fa37bae4b6c98c38f5be
  Preparing metadata (setup.py) ... done
  Using cached pytorchvideo-0.1.5-py3-none-any.whl


In [3]:
import os
import glob
import torch

from imagebind.models import imagebind_model
from imagebind.models.imagebind_model import ModalityType
from imagebind import data as imagebind_data

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


In [4]:
# Create folder for audio files
audio_dir = "/content/audio"
os.makedirs(audio_dir, exist_ok=True)

from google.colab import files
uploaded = files.upload()  # select a few .wav files

# Move uploaded files into /content/audio
for fname in uploaded.keys():
    os.replace(fname, os.path.join(audio_dir, fname))

print("Uploaded files:", os.listdir(audio_dir))


Saving file_example_WAV_1MG.wav to file_example_WAV_1MG.wav
Uploaded files: ['file_example_WAV_1MG.wav']


In [5]:
audio_dir = "/content/audio"  # change if needed

audio_paths = sorted(glob.glob(os.path.join(audio_dir, "*.wav")))
print("Found", len(audio_paths), "audio clips")

if len(audio_paths) == 0:
    raise ValueError("No audio files found. Put some .wav files into /content/audio first.")


Found 1 audio clips


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

model = imagebind_model.imagebind_huge(pretrained=True)
model.eval()
model.to(device)


Using device: cuda


ImageBindModel(
  (modality_preprocessors): ModuleDict(
    (vision): RGBDTPreprocessor(
      (cls_token): tensor((1, 1, 1280), requires_grad=True)
      
      (rgbt_stem): PatchEmbedGeneric(
        (proj): Sequential(
          (0): PadIm2Video()
          (1): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
        )
      )
      (pos_embedding_helper): SpatioTemporalPosEmbeddingHelper(
        (pos_embed): tensor((1, 257, 1280), requires_grad=True)
        
      )
    )
    (text): TextPreprocessor(
      (pos_embed): tensor((1, 77, 1024), requires_grad=True)
      (mask): tensor((77, 77), requires_grad=False)
      
      (token_embedding): Embedding(49408, 1024)
    )
    (audio): AudioPreprocessor(
      (cls_token): tensor((1, 1, 768), requires_grad=True)
      
      (rgbt_stem): PatchEmbedGeneric(
        (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10), bias=False)
        (norm_layer): LayerNorm((768,), eps=1e-05, elementwise_affine=

In [9]:
!pip install torchcodec


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.0 MB/s eta 0:00:00


In [10]:
# Load and transform audio with the correct helper
audio_inputs = imagebind_data.load_and_transform_audio_data(
    audio_paths,
    device,
)

with torch.no_grad():
    outputs = model({ModalityType.AUDIO: audio_inputs})

audio_emb = outputs[ModalityType.AUDIO].cpu().numpy()
print("Embeddings shape:", audio_emb.shape)


Embeddings shape: (1, 1024)


In [11]:
# choose number of clusters (e.g., 2 or 3 depending on how many different types of sounds you have)
k = 2 if len(audio_paths) >= 2 else 1

kmeans = KMeans(n_clusters=k, random_state=42)
labels = kmeans.fit_predict(audio_emb)

if len(set(labels)) > 1:
    sil = silhouette_score(audio_emb, labels)
    print(f"Silhouette score: {sil:.3f}")

# Print which files fall into which cluster
for c in range(k):
    print(f"\nCluster {c}:")
    for path, lab in zip(audio_paths, labels):
        if lab == c:
            print("  ", os.path.basename(path))



Cluster 0:
   file_example_WAV_1MG.wav
